##### Copyright 2024 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Similarity Search using Gemini API and Qdrant

## Overview



The [Gemini API](https://ai.google.dev/models/gemini) provides access to a family of generative AI models for generating content and solving problems. These models are designed and trained to handle both text and images as input.

[Qdrant](https://qdrant.tech/) is a vector similarity search engine that offers an easy-to-use API for managing, storing, and searching vectors, with an additional payload. It is a production-ready service.

In this notebook, you'll learn how to perform a similarity search on data from a website with the help of Gemini API and Qdrant.

## Setup

First, you must install the packages and set the necessary environment variables.

### Installation

Install google's python client SDK for the Gemini API, `google-generativeai`.

In [2]:
! pip install -q google-generativeai

Install Qdrant's python client SDK, `qdrant-client`.

In [ ]:
! pip install -q qdrant-client

### Grab and set the API key

To use Gemini API you need an *API key*. You can create an API key with one click in [Google AI Studio](https://makersuite.google.com/).

Once you have the API key, pass it to the SDK. You can do this in two ways:

1. Assign the key to the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there) or pass the key to `genai.configure(api_key=...)`.
2. Or provide it explicitly through the `api_key` parameter.

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) guide for an example.

In [ ]:
import google.generativeai as genai

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Basic steps

Semantic search is the process using which search engines interpret and match keywords to a user's intent in organic search results. It goes beyond surface-level keyword matching. It uses the meaning of words, phrases, and context using advanced algorithms resulting in more relevant and user-friendly search experiences.

Semantic searches rely on vector embeddings which can best match the user query to the most similar result.

In this tutorial, you'll implement the three main components of semantic search:

1. Create an index

    Create and store the index for the data in the Qdrant vector store. You will use a Gemini API embedding model to create embedding vectors that can be stored in the Qdrant vector store.

2. Query the index

    Query the index using a query string to return the top `n` neighbors of the query.

You'll learn more about these stages in the upcoming sections while implementing the application.

## Import the required libraries

In [ ]:
from bs4 import BeautifulSoup
from qdrant_client import models, QdrantClient
from urllib.request import urlopen

## 1. Create an index

In this stage, you will perform the following steps:

1. Read and parse the website data using Python's BeautifulSoup library.

2. Create embeddings of the website data.

3. Store the embeddings in Qdrant's vector database.
    
    Qdrant is a vector similarity search engine. Along with a convenient API to store, search, and manage points(i.e. vectors), it also provides an option to add an additional payload. The payloads are essentially extra bits of data that you can utilize to refine your search and obtain relevant information that you can then share with your users.

### Read and parse the website data

To read the website data as text, you will use the `BeautifulSoup` library from Python.

In [ ]:
url = "https://blog.google/outreach-initiatives/sustainability/"\
      "report-ai-sustainability-google-cop28/"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# Remove all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # Self-destruct

# Get the text
text_content = soup.get_text()

If you only want to select a specific portion of the website data to add context to the prompt, you can use regex, text slicing, or text splitting.

In this example, you'll use Python's `split()` function to extract the required portion of the text.

In [ ]:
# The text content between the substrings "Later this month at COP28" to
# "POSTED IN:" is relevant for this tutorial. You can use Python's `split()`
# to select the required content.
text_content_1 = text_content.split("Later this month at COP28",1)[1]
final_text = text_content_1.split("POSTED IN:",1)[0]

texts = final_text.split(".")

documents = []

# Convert text into a chunk of 3 sentences.
for i in range(0, len(texts), 3):
  documents.append({"content": " ".join(texts[i:i+3])})

### Initialize the embedding model

To create the embeddings from the website data, you'll use the **embedding-001** model, which supports creating embeddings from text.

To use the embedding model, you have to use the `embed_content` function from the `google-generativeai` package. To learn more about the embedding model, read the [model documentation](https://ai.google.dev/gemini-api/docs/models/gemini#embedding).

One of the arguments passed to the embedding function is `task_type`. Speciefying the `task_type` parameter ensures the model produces appropriate embeddingsfor the expected task and inputs. It is a string that can take on one of the following values:

| task_type	  |  Description |
|---|---|
| `RETRIEVAL_QUERY` | Specifies the given text is a query in a search or retrieval setting. |
| `RETRIEVAL_DOCUMENT` | Specifies the given text is a document in a search or retrieval setting. |  
| `SEMANTIC_SIMILARITY` | Specifies the given text will be used for Semantic Textual Similarity (STS). |  
| `CLASSIFICATION` | Specifies that the embeddings will be used for classification. |
| `CLUSTERING` | Specifies that the embeddings will be used for clustering. |

In [ ]:
# Default embedding model
embedding_model = "models/embedding-001"

# Function to convert text to embeddings
def make_embed_text_fn(text, model=embedding_model,
                       task_type="retrieval_document"):
    embedding = genai.embed_content(model=model,
                                    content=text,
                                    task_type=task_type)
    return embedding['embedding']

### Store the data using Qdrant

 Next, you'll store the embeddings of the website data in Qdrant's vector store.

 First, you have to initiate a Qdrant client by creating an instance of `QdrantClient`. In this tutorial, you will store the embeddings in memory. To create an in-memory Qdrant client specify `:memory:` for the `location` argument of the `QdrantClient` class initializer. You can read more about the different types of storage in Qdrant in the [storage reference guide](https://qdrant.tech/documentation/concepts/storage/).

After initializing the client, you have to create a Qdrant collection using the `recreate_collection` function of `QdrantClient`. You can specify your vector configuration inside the `recreate_collection` function. Pass an instance of `VectorParams` with the `size` set to `768` to match the embedding model and `distance` set to cosine.

**Note**: Since you will run the script several times during your experiments, `recreate_collection` is appropriate for this tutorial. `recreate_collection` will first try to remove an existing collection with the same name.

In [ ]:
# Initialize Qdrant client.
qdrant = QdrantClient(":memory:")

# Create a collection named "GeminiCollection".
qdrant.recreate_collection(
    collection_name="GeminiCollection",
    vectors_config=models.VectorParams(
        size=768,  # Vector size of `embedding-001`
        distance=models.Distance.COSINE,
    ),
)

You will now insert the `documents` you parsed from the website data into the Qdrant collection you created earlier and index them using the `upsert` function of `QdrantClient`.

The `upsert` function takes the data to be stored and indexed as an array of `PointsStruct`s.

Points are the main entity in Qdrant operations. A point is a record consisting of a vector and an optional payload. You can perform a similarity search among the points in one collection. Read more about points in [Qdrant's points documentation](https://qdrant.tech/documentation/concepts/points/).

You'll create an array of points by enumerating over the documents you prepared earlier from the website data.

In [ ]:
# Qdrant uses batch loading of points to optimize performance.
# You can create a batch in two ways - record-oriented and column-oriented.
# Here you are using the record-oriented approach.

qdrant.upsert(
    collection_name="GeminiCollection",
    points=[
        # Use PointStruct function to intialize the point.
        models.PointStruct(
            # Use `make_embed_text_fn` to convert text to embeddings.
            # Pass the same data as payload for a refined search.
            id=idx, vector=make_embed_text_fn(doc["content"]), payload = doc
        )
        for idx, doc in enumerate(documents)
    ]
)

## 2. Query the index

You'll now query the Qdrant index you created earlier with a question related to the data contained in the website documents.
To query the index, you have to mention the collection name and the query vector. The query vector should be first converted to an embedding vector using the Gemini API embedding model you leveraged to create embedding vectors for the website data. Use the `make_embed_text_fn` you defined earlier for creating an embedding vector from your query. Since you are embedding a query string that is being used to search `retrieval_document` embeddings, the `task_type` must be set to `retrieval_query`.

In [ ]:

hits = qdrant.search(
    collection_name="GeminiCollection",
    query_vector=make_embed_text_fn("How can AI address climate challenges?",
                                    task_type="retrieval_query"),
    limit=3,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

##Conclusion

That's it. You have successfully performed a similarity search using Qdrant with the help of a Gemini API embedding model.